In [3]:
from dataset import TRANSFORMER_ALL_DATA, TRANSFORMERDATA
from loss import advLoss
from model import *
from torch.utils.data import DataLoader, random_split
import argparse
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import time
import torch
import torch.nn as nn
from tqdm import tqdm
# def my own train
seed = 0
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed)       
torch.cuda.manual_seed_all(seed)    
random.seed(seed)
np.random.seed(seed)
Rc = 130

data_root = "CMAPSS/units/"
label_root = "CMAPSS/labels/"
seq_len = 70
target= 'FD002'
source = 'FD003'
epoches = 240
os.chdir('/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/')
batch_size = 100
a = 0.1
b = 0.5

In [4]:
#  compare data
source_list = np.loadtxt("save/"+source+"/train"+source+".txt", dtype=str).tolist()
target_list = np.loadtxt("save/"+target+"/train"+target+".txt", dtype=str).tolist()
valid_list = np.loadtxt("save/"+target+"/test"+target+".txt", dtype=str).tolist()
# v_list with 47 files
a_list = np.loadtxt("save/"+target+"/valid"+target+".txt", dtype=str).tolist()
# a_list with 31 item of validation set item
target_test_names = valid_list + a_list
# 78 files in targetlist....
minl = min(len(source_list), len(target_list))
s_data = TRANSFORMER_ALL_DATA(source_list, seq_len)
t_data = TRANSFORMER_ALL_DATA(target_list, seq_len)
t_data_test = TRANSFORMER_ALL_DATA(target_test_names, seq_len)

In [5]:
len(s_data)
l= t_data[0]
for feature in l:
    print(Tensor.mean(feature))
    print(Tensor.var(feature))
    print(feature)
len(l[2])

tensor(0.0097)
tensor(0.0072)
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2380, 0.2971, 1.0000,  ..., 1.0000, 0.6289, 0.6289]])
tensor(1.)
tensor(0.)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor(0.9857)
tensor(0.0143)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

70

In [6]:
source_list[0]

'FD003-56.txt'

In [7]:
#  check detail of dataset
batch_size = 100
s_iter = iter(DataLoader(s_data, batch_size=batch_size, shuffle=False))
t_iter = iter(DataLoader(t_data, batch_size=batch_size, shuffle=False))

In [20]:
len(s_iter)

215

In [21]:
len(s_data)

21489

In [8]:
s_d = next(s_iter)
t_d = next(t_iter)

In [9]:
print(len(s_d))
print(len(s_d[2]))
print(s_d[2].shape)
# print((s_d[0][0] - sf).sum())
type(s_d[2])

3
100
torch.Size([100, 70])


torch.Tensor

In [10]:
type(s_data)

dataset.TRANSFORMER_ALL_DATA

In [11]:
# test model output
net = mymodel(max_len=seq_len)
sf,sm,sl = s_data[0]

In [12]:
len(sf)
sf[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:

sf = sf.unsqueeze(0)
sl = sl.unsqueeze(0)
sm = sm.unsqueeze(0)
o1,o2 = net(sf,sm)

In [14]:
o1.shape

torch.Size([1, 70, 24])

In [15]:
o2

tensor([[[ 3.7776e-02],
         [ 2.0524e-01],
         [ 3.1993e-01],
         [ 2.6842e-01],
         [ 5.6817e-01],
         [ 1.0271e-01],
         [-2.8446e-02],
         [ 2.3681e-01],
         [-1.5536e-01],
         [ 1.0356e-01],
         [ 1.3156e-01],
         [ 2.0401e-02],
         [ 2.5028e-01],
         [ 1.2599e-01],
         [ 4.0941e-01],
         [ 2.4081e-01],
         [ 5.2443e-01],
         [ 2.5499e-01],
         [ 1.5482e-01],
         [ 6.3721e-01],
         [ 4.2785e-01],
         [ 3.9672e-01],
         [ 3.9084e-01],
         [ 4.3673e-01],
         [ 3.6688e-01],
         [ 1.4496e-01],
         [ 2.6291e-01],
         [ 1.5641e-01],
         [ 5.6852e-01],
         [ 4.7181e-01],
         [ 2.6387e-01],
         [-8.6811e-02],
         [ 9.1588e-02],
         [-3.5293e-01],
         [-1.3847e-01],
         [-1.1308e-01],
         [-2.7357e-02],
         [-1.0959e-01],
         [-1.7909e-01],
         [-2.6981e-01],
         [-5.1335e-02],
         [ 9.356

In [16]:
o1.shape

torch.Size([1, 70, 24])

In [17]:
o1

tensor([[[-1.6653, -0.0533, -0.0469,  ...,  0.9081, -0.3176,  0.8180],
         [-0.4419, -1.5840,  0.0000,  ...,  1.3871, -0.0892,  0.9407],
         [-1.6055, -1.8214,  0.8913,  ..., -0.4419,  0.1452,  0.6301],
         ...,
         [-2.4871, -1.6600, -0.6493,  ...,  0.8330, -0.2036,  0.0000],
         [-2.6875, -0.4647,  1.1753,  ...,  0.2619, -0.6457,  1.6955],
         [-1.6099, -0.0000,  0.0000,  ...,  1.2234, -1.2426,  1.2773]]],
       grad_fn=<MulBackward0>)

In [18]:
Tensor.mean(o1)

tensor(0.0062, grad_fn=<MeanBackward0>)

In [19]:
Tensor.var(o1)

tensor(1.1252, grad_fn=<VarBackward0>)

In [30]:
net.training

True

In [ ]:

def validate():
    net.eval()
    tot = 0
    with torch.no_grad():
        for i in target_test_names:
            pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
            # 记录结果 （800维？）
            valid_data = TRANSFORMERDATA(i, seq_len)
            # 变换一项结果
            data_len = len(valid_data)
            valid_loader = DataLoader(valid_data, batch_size=1000)
            # 参考train中dataloader进行改造
            valid_iter = iter(valid_loader)
            d = next(valid_iter)
            # 迭代
            input, lbl, msk = d[0], d[1], d[2]
            input, msk = input.cuda(), msk.cuda()
            _, out = net(input, msk)
            # 输出结果，out 就是decoder的输出
            out = out.squeeze(2).cpu()
            for j in range(data_len):
                if j < seq_len-1:
                    pred_sum[:j+1] += out[j, -(j+1):]
                    pred_cnt[:j+1] += 1
                elif j <= data_len-seq_len:
                    pred_sum[j-seq_len+1:j+1] += out[j]
                    pred_cnt[j-seq_len+1:j+1] += 1
                else:
                    pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
                    pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
            truth = torch.tensor([lbl[j,-1] for j in range(len(lbl)-seq_len+1)], dtype=torch.float)
            pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
            pred = pred_sum/pred_cnt
            mse = float(torch.sum(torch.pow(pred-truth, 2)))
            rmse = math.sqrt(mse/data_len)
            tot += rmse
    return tot*Rc/len(valid_list)


def train():
    minn = 999
    for e in tqdm(range(epochs)):
        al, tot = 0, 0
        net.train()
        random.shuffle(source_list)
        random.shuffle(target_list)
        source_iter, target_iter = iter(source_list), iter(target_list)
        loss2_sum, loss1_sum = 0, 0
        bkb_sum, out_sum = 0, 0
        cnt = 0
        s_iter = iter(DataLoader(s_data, batch_size=args.batch_size, shuffle=True))
        t_iter = iter(DataLoader(t_data, batch_size=args.batch_size, shuffle=True))
        l = min(len(s_iter), len(t_iter))
        for _ in range(l):
            s_d, t_d = next(s_iter), next(t_iter)
            s_input, s_lb, s_msk = s_d[0], s_d[1], s_d[2]
            t_input, t_msk = t_d[0], t_d[2]
            s_input, s_lb, s_msk = s_input.cuda(), s_lb.cuda(), s_msk.cuda()
            t_input, t_msk = t_input.cuda(), t_msk.cuda()
            s_features, s_out = net(s_input, s_msk)
            t_features, t_out = net(t_input, t_msk) # [bts, seq_len, feature_num]
            s_out.squeeze_(2)
            t_out.squeeze_(2)
            loss1 = Loss(s_out, s_lb)
            loss1_sum += loss1
            cnt += 1
            if args.type == 1 or args.type == 0:
                if args.type == 1:
                    s_domain = D2(s_features)
                    t_domain = D2(t_features)
                else:
                    s_domain = D1(s_out)
                    t_domain = D1(t_out)
                loss2 = advLoss(s_domain.squeeze(1), t_domain.squeeze(1), 'cuda')
                loss2_sum += loss2
                loss = loss1 + a*loss2
            elif args.type == 2:
                s_domain_bkb = D2(s_features)
                t_domain_bkb = D2(t_features)
                s_domain_out = D1(s_out)
                t_domain_out = D1(t_out)
                if e>=5:
                    fea_loss = advLoss(s_domain_bkb.squeeze(1), t_domain_bkb.squeeze(1), 'cuda')
                    out_loss = advLoss(s_domain_out.squeeze(1), t_domain_out.squeeze(1), 'cuda')
                    bkb_sum += fea_loss
                    out_sum += out_loss
                    loss = loss1 + a*fea_loss + b*out_loss
                else:
                    loss = loss1
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(itertools.chain(net.parameters(), D1.parameters(), D2.parameters()), 2)
            opt.step()    

        rmse = validate()
        if args.type == 2:
            print("{}/{}| loss1={:.5f}, fea_loss={:.5f}, out_loss={:.5f}, rmse={:.5f}".\
                format(e, args.epoch, loss1_sum/cnt, bkb_sum/cnt, out_sum/cnt, rmse))
            
            with open("/Domain-Adaptive-Remaining-Useful-Life-Prediction-with-Transformer/result.txt",'a') as f:
                f.write("{}/{}| loss1={:.5f}, fea_loss={:.5f}, out_loss={:.5f}, rmse={:.5f}\n".\
                format(e, args.epoch, loss1_sum/cnt, bkb_sum/cnt, out_sum/cnt, rmse))
        else:    
            print("{}/{}| 1={:.5f}, 2={:.5f}, rmse={:.5f}".format(e, args.epoch, loss1, loss2_sum/cnt, rmse))
        if rmse<minn:
            minn = rmse
            print("min={}".format(minn))
            if args.type == 1:
                torch.save(net.state_dict(), "save/final/dann_"+source[-1]+target[-1]+".pth")
            elif args.type == 0:
                torch.save(net.state_dict(), "save/final/out_"+source[-1]+target[-1]+".pth")
            elif args.type == 2 :
                #torch.save(net.state_dict(), "save/final/both_"+source[-1]+target[-1]+".pth")
                torch.save(net.state_dict(), "online/"+source[-1]+target[-1]+"_net.pth")
                torch.save(D1.state_dict(), "online/"+source[-1]+target[-1]+"_D1.pth")
                torch.save(D2.state_dict(), "online/"+source[-1]+target[-1]+"_D2.pth")
        
        if args.scheduler:
            sch.step()

    return minn

ModuleNotFoundError: No module named 'mydataset'